<a href="https://colab.research.google.com/github/YNNJN/colorPOOL_Data/blob/master/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. 데이터 로드 & 상위 데이터 추출

- 각 배색에 대한 조화도 평점을 기준으로, 내림차순 정렬하여 상위 3000개를 데이터 프레임으로 추출 완료함
    - 상위 3000개 데이터의 조화도 평균 평점 : 4.35, 표준편차 : 0.26

In [1]:
!git clone https://github.com/YNNJN/colorPool_Data.git

Cloning into 'colorPool_Data'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 64 (delta 34), reused 15 (delta 3), pack-reused 0
Unpacking objects: 100% (64/64), done.


In [2]:
!ls -ltr

total 8
drwxr-xr-x 1 root root 4096 Sep 16 16:29 sample_data
drwxr-xr-x 4 root root 4096 Sep 28 08:18 colorPool_Data


In [3]:
# connect to google drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

rgbData = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/kulerData_data.csv', names=['R1', 'R2', 'R3', 'R4', 'R5', 'G1', 'G2', 'G3', 'G4', 'G5', 'B1', 'B2', 'B3', 'B4', 'B5' ])
ratingData = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/kulerData_targets.csv', names=['rating'])
idData = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/kulerData_ids.csv', names=['id'])
nameData = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/kulerData_name.csv', names=['name'])

rgbRatingData = pd.concat([rgbData, ratingData, idData, nameData], axis=1)

In [5]:
df = pd.DataFrame(rgbRatingData)
df_sorted = df.sort_values(by=['rating'], ascending=[False])
df_extracted = df_sorted[0:3000]
df_rgbExtracted = df_extracted.drop(df_extracted.columns[-1], axis=1)
df_rgbExtracted = df_rgbExtracted.drop(df_extracted.columns[-2], axis=1)
df_rgbExtracted = df_rgbExtracted.drop(df_extracted.columns[-3], axis=1)

df_rgbExtracted

# df_extracted.to_csv('gdrive/My Drive/colorPOOL/kulerData/matrixData.csv')

,R1,R2,R3,R4,R5,G1,G2,G3,G4,G5,B1,B2,B3,B4,B5
44985,0.43922,0.619610,0.45490,0.27843,0.388240,0.286270,0.52157,0.47843,0.27451,0.30196,0.172550,0.270590,0.341180,0.196080,0.16471
37404,0.05098,0.949020,0.74902,0.68235,0.643140,0.050980,0.94902,0.85098,0.85098,0.85098,0.050980,0.949020,0.721570,0.639220,0.58824
36535,0.93333,0.850980,0.67059,0.42745,0.317650,0.568630,0.33333,0.74902,0.65098,0.55686,0.058824,0.145100,0.082353,0.090196,0.65098
4099,0.76863,0.039216,0.80392,0.40000,0.000000,0.000000,0.50980,0.00000,0.25490,0.00000,0.000000,0.756860,0.509800,0.125490,0.00000
4091,1.00000,0.823530,0.88627,0.67059,0.403920,0.984310,0.76863,0.44314,0.32157,0.21569,0.839220,0.725490,0.215690,0.196080,0.13725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31263,0.80000,0.980390,0.67451,0.34510,0.164710,0.325490,1.00000,1.00000,0.80000,0.52549,0.513730,0.580390,0.874510,0.623530,0.60000
31270,0.65098,0.949020,0.94902,0.15294,0.372550,0.545100,0.94902,0.91373,0.17255,0.45098,0.011765,0.949020,0.427450,0.188240,0.40000
31278,0.85098,0.850980,0.94902,0.47451,0.011765,0.015686,0.40392,0.62353,0.65098,0.58824,0.015686,0.015686,0.019608,0.090196,0.65098
31279,0.14902,0.549020,0.85098,0.94902,0.949020,0.003922,0.15294,0.41569,0.56863,0.88235,0.003922,0.011765,0.160780,0.239220,0.55294


# 1. 표본 색채 설정 & 색차 허용치 보정
- 표본 색채 설정 이유
    - Kuler에서 활용되는 색채들 중 서로 비슷한 색들을 근사하여 분석하기 위해, 인지적으로 균등한 표본 색채를 설정함
- 색차 허용치 보정 이유
    - 색차 표본간의 거리를 사람이 인식하는 색차와 균등하게 보정하기 위함

#### McDonald의 색차 계산식 이용
- 이유 : CIE L*a*b 색공간 내에서의 기하학적 거리 ≠ 사람의 인지적 색차
    - 방법 : 색차허용치 = 색공간 내 기하학적 거리/(1+0.022(채도값)) = 6.35
    - 의의 : 색채 표본간의 거리를 사람이 인식하는 색차와 균등하게 보정 완료
- 진행과정 :

## 1) 데이터 변환
- RGB → XYZ → CIE L*a*b

In [6]:
# 데이터프레임 구조 변환 3000*15 => 15000*3

df1=df_rgbExtracted.iloc[:,[0,5,10]].reset_index(drop=True).rename(columns={"R1":'R',"G1":'G',"B1":'B'})
df2=df_rgbExtracted.iloc[:,[1,6,11]].reset_index(drop=True).rename(columns={"R2":'R',"G2":'G',"B2":'B'})
df3=df_rgbExtracted.iloc[:,[2,7,12]].reset_index(drop=True).rename(columns={"R3":'R',"G3":'G',"B3":'B'})
df4=df_rgbExtracted.iloc[:,[3,8,13]].reset_index(drop=True).rename(columns={"R4":'R',"G4":'G',"B4":'B'})
df5=df_rgbExtracted.iloc[:,[4,9,14]].reset_index(drop=True).rename(columns={"R5":'R',"G5":'G',"B5":'B'})

df_all=pd.concat([df1,df2,df3,df4,df5], axis=0).reset_index(drop=True)
df_all

,R,G,B
0,0.439220,0.28627,0.172550
1,0.050980,0.05098,0.050980
2,0.933330,0.56863,0.058824
3,0.768630,0.00000,0.000000
4,1.000000,0.98431,0.839220
...,...,...,...
14995,0.164710,0.52549,0.600000
14996,0.372550,0.45098,0.400000
14997,0.011765,0.58824,0.650980
14998,0.949020,0.88235,0.552940


In [7]:
df_all = df_all * 255.0
df_all

,R,G,B
0,112.001100,72.99885,44.00025
1,12.999900,12.99990,12.99990
2,237.999150,145.00065,15.00012
3,196.000650,0.00000,0.00000
4,255.000000,250.99905,214.00110
...,...,...,...
14995,42.001050,133.99995,153.00000
14996,95.000250,114.99990,102.00000
14997,3.000075,150.00120,165.99990
14998,242.000100,224.99925,140.99970


In [8]:
!pip install colormath

  Created wheel for colormath: filename=colormath-3.0.0-cp36-none-any.whl size=39392 sha256=651cada75c5d59981755430809f377d81b03e595b953757ec4a518b382a5f4b9
  Stored in directory: /root/.cache/pip/wheels/20/49/7b/9635bccd6136cc713f623caa85ced89c0c63878f8b2a58fe99
Successfully built colormath


In [9]:
# RGB → Lab

from colormath.color_objects import LabColor, sRGBColor 
from colormath.color_conversions import convert_color 

l=[]
a=[]
b=[]

for i in range(len(df_all)) :
    rgb = sRGBColor(df_all['R'][i], df_all['G'][i], df_all['B'][i], is_upscaled=True)
    lab = convert_color(rgb, LabColor, through_rgb_type=sRGBColor)
    l.append(lab.lab_l)
    a.append(lab.lab_a)
    b.append(lab.lab_b)

In [10]:
lab = pd.DataFrame({"L":l,"a":a,"b":b})
lab

,L,a,b
0,34.786683,13.291604,23.964126
1,3.635461,-0.000043,-0.000805
2,68.362727,27.425410,71.602494
3,40.798164,65.699870,55.126697
4,98.077292,-4.623466,18.233872
...,...,...,...
14995,51.656988,-20.060466,-17.912878
14996,46.516427,-10.225682,4.739290
14997,56.667610,-27.024131,-17.807196
14998,89.229635,-5.618007,43.125657


## 2) 데이터 양자화


In [11]:
# quantize

def quant(x):
    if x % 5 == 0:
        return int(x)
    else:
        return int(int((x + 5) / 5) * 5)

quant_l = []
quant_a = []
quant_b = []

for i in range(len(lab)):
    quant_l.append(quant(lab.L[i]))
    quant_a.append(quant(lab.a[i]))
    quant_b.append(quant(lab.b[i]))

In [12]:
quant_lab = pd.DataFrame({"L" : quant_l, "a" : quant_a, "b": quant_b})
quant_lab

,L,a,b
0,35,15,25
1,5,0,0
2,70,30,75
3,45,70,60
4,100,0,20
...,...,...,...
14995,55,-15,-10
14996,50,-5,5
14997,60,-20,-10
14998,90,0,45


In [13]:
!pip install -q colour-science

     |████████████████████████████████| 1.6MB 2.7MB/s 


In [14]:
# 적정 dEh 값과 비교하기 위한 테스트 진행
# Cab = (a ** 2 + b ** 2) ** 1/2
# dEh = dE / (1 + 0.022 * Cab)

import colour

dE = []

for i in range(len(lab)):
    previous = [lab.L[i], lab.a[i], lab.b[i]]
    next = [quant_lab.L[i], quant_lab.a[i], quant_lab.b[i],]

    de = colour.delta_E(previous, next, method='CIE 2000')
    dE.append(de)

In [15]:
cab = []

for i in range(len(lab)):
    cal = (lab.a[i] ** 2 + lab.b[i] ** 2) ** 1/2
    cab.append(cal)

In [16]:
tmp = pd.DataFrame({"dE": dE, "Cab" : cab})
tmp

,dE,Cab
0,1.151602,3.754730e+02
1,0.811273,3.249352e-07
2,1.708097,2.939535e+03
3,4.091402,3.677713e+03
4,5.311534,1.769253e+02
...,...,...
14995,5.903835,3.616468e+02
14996,6.280096,6.351273e+01
14997,5.832186,5.237000e+02
14998,4.083315,9.456922e+02


In [17]:
deh = []
for i in range(len(tmp)):
    cal = tmp.dE[i] / (1 + 0.022 * tmp.Cab[i])
    deh.append(cal)

In [18]:
deh = pd.DataFrame(data=deh, columns=['dEh'])
deh

,dEh
0,0.124358
1,0.811273
2,0.026010
3,0.049950
4,1.085680
...,...
14995,0.659188
14996,2.619676
14997,0.465778
14998,0.187263


In [19]:
# 평균
deh.mean() # 1.192859

dEh    1.192859
dtype: float64

In [20]:
# 분산
deh.var() # 2.418498

dEh    2.418498
dtype: float64

## 3) 데이터 양자화 2
- 양자화 간격을 3으로 조정함
    - dE의 값을 5 이하로 하는 dL, da, db 의 값을 추산한 결과로 간격인 3을 결정함
- 인지적으로 균등한 색채 이론 참고
    - [Perceptual Color Space](http://cs.haifa.ac.il/hagit/courses/ist/Lectures/IST05_ColorLAB.pdf)
- 양자화 후 dEh 값을 역으로 계산하여 양자화 신뢰도 검증함
    - dEh 값은 평균 0.792064, 분산은 1.181968
    - 데이터의 신뢰도 검증 완료
- 양자화 한 색채로 테마를 구성했을 때의 시각적 유사도 또한 육안으로 검증하는 과정을 거쳤음

In [21]:
# quantize

def quant2(x):
    if x % 3 == 0:
        return int(x)
    else:
        return int(int((x + 3) / 3) * 3)

quant2_l = []
quant2_a = []
quant2_b = []

for i in range(len(lab)):
    quant2_l.append(quant2(lab.L[i]))
    quant2_a.append(quant2(lab.a[i]))
    quant2_b.append(quant2(lab.b[i]))

In [22]:
quant2_lab = pd.DataFrame({"L" : quant2_l, "a" : quant2_a, "b": quant2_b})
quant2_lab
quant2_lab.to_csv('gdrive/My Drive/colorPOOL/kulerData/quantize.csv')

In [23]:
dE = []

for i in range(len(lab)):
    previous = [lab.L[i], lab.a[i], lab.b[i]]
    next = [quant2_lab.L[i], quant2_lab.a[i], quant2_lab.b[i],]

    de = colour.delta_E(previous, next, method='CIE 2000')
    dE.append(de)

In [24]:
cab = []

for i in range(len(lab)):
    cal = (lab.a[i] ** 2 + lab.b[i] ** 2) ** 1/2
    cab.append(cal)

In [25]:
tmp = pd.DataFrame({"dE": dE, "Cab" : cab})
tmp

,dE,Cab
0,1.642781,3.754730e+02
1,1.412139,3.249352e-07
2,1.452595,2.939535e+03
3,1.289372,3.677713e+03
4,5.338274,1.769253e+02
...,...,...
14995,4.581062,3.616468e+02
14996,4.622651,6.351273e+01
14997,3.290195,5.237000e+02
14998,4.083315,9.456922e+02


In [26]:
deh = []
for i in range(len(tmp)):
    cal = tmp.dE[i] / (1 + 0.022 * tmp.Cab[i])
    deh.append(cal)

In [27]:
deh2 = pd.DataFrame(data=deh, columns=['dEh'])
deh2

,dEh
0,0.177398
1,1.412139
2,0.022120
3,0.015741
4,1.091146
...,...
14995,0.511495
14996,1.928290
14997,0.262766
14998,0.187263


In [28]:
# 평균
deh2.mean() # 0.792064

dEh    0.792064
dtype: float64

In [29]:
# 분산
deh2.var() # 1.181968

dEh    1.181968
dtype: float64